In [79]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [67]:
with open('wizard_of_oz.txt' , 'r', encoding='utf-8') as f:
    text = f.read()
#print(text[:200])

chars = sorted(set(text))
print (chars)
vocab_size= len(chars)
vocab_size

['\n', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


89

In [68]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

#encoded_hello = encode('hello')
#decoded_hello = decode(encoded_hello)
#decoded_hello

data = torch.tensor(encode(text), dtype=torch.long)
print (data[:100])

tensor([88, 46, 63, 74, 66, 59, 24,  1, 46, 62, 59,  1, 49, 69, 68, 58, 59, 72,
        60, 75, 66,  1, 49, 63, 80, 55, 72, 58,  1, 69, 60,  1, 41, 80,  0,  0,
         0, 27, 75, 74, 62, 69, 72, 24,  1, 38, 12,  1, 32, 72, 55, 68, 65,  1,
        28, 55, 75, 67,  0,  0, 44, 59, 66, 59, 55, 73, 59,  1, 58, 55, 74, 59,
        24,  1, 32, 59, 56, 72, 75, 55, 72, 79,  1, 15, 10,  1, 15, 23, 23, 17,
         1, 53, 59, 28, 69, 69, 65,  1,  3, 19])


In [69]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[84, 35,  1, 77, 63, 66, 66,  1],
        [62, 59,  1, 64, 69, 75, 72, 68],
        [61,  1, 74, 62, 59, 79,  1, 57],
        [ 1, 63, 73,  1, 74, 62, 59,  1]], device='cuda:0')
targets:
tensor([[35,  1, 77, 63, 66, 66,  1, 61],
        [59,  1, 64, 69, 75, 72, 68, 59],
        [ 1, 74, 62, 59, 79,  1, 57, 55],
        [63, 73,  1, 74, 62, 59,  1, 73]], device='cuda:0')


In [70]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([88]) target is tensor(46)
when input is tensor([88, 46]) target is tensor(63)
when input is tensor([88, 46, 63]) target is tensor(74)
when input is tensor([88, 46, 63, 74]) target is tensor(66)
when input is tensor([88, 46, 63, 74, 66]) target is tensor(59)
when input is tensor([88, 46, 63, 74, 66, 59]) target is tensor(24)
when input is tensor([88, 46, 63, 74, 66, 59, 24]) target is tensor(1)
when input is tensor([88, 46, 63, 74, 66, 59, 24,  1]) target is tensor(46)


In [71]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [77]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:        
            #batch, time, channel
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


CcI﻿jE0—t/g”7who[d—﻿Kr$G”&:pZZ“r2XTCdstaM]qNs#6Xa“Zq]™#L;L(0q/”&Tj:v﻿bpBejZw.UluB9,h%‘Ma5P‘hK#’iz2‘*U-—tR‘aMaJ’—K#f—Bpn3HeDkDoslzhGZNj‘h#8i‘hW7NQW!k3ZZuTn-:ggYrUj‘•)H: ,!7C64T8P9M
Hys3fId%LG3w,S(wn﻿DQ3Zpw6Xun2“2]-”8*R,QZZ8S!hz-)QBJIJo-qNsv%uti5™YeG.i‘•Y%o7wh]f8PwjB?6G!?un‘c‘,iz—$f/oxxi5M),PT-9T&b“™[Li‘zj!pF;pw4L nfa.i—!$Aujo-WczPZe3fAMh]k—x;$FrGiza[RX:mi2r?ZYMASN&]QxL•zkCYC;]/.-gnNysXBJ%9Lpfz?-﻿BG:﻿[’o6kM8“Ir:sd)‘Px5Q‘soc5tX0•h;Y’ryB2 vN2l(a0/Q:L( 32$f—GET‘w6FfR1UP-Av!#PgK‘/jsBebLfS2]4l‘lVoG:EIg


In [80]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.708, val loss: 4.768
step: 250, train loss: 4.630, val loss: 4.698
step: 500, train loss: 4.601, val loss: 4.641
step: 750, train loss: 4.513, val loss: 4.591
step: 1000, train loss: 4.459, val loss: 4.514
step: 1250, train loss: 4.405, val loss: 4.481
step: 1500, train loss: 4.342, val loss: 4.401
step: 1750, train loss: 4.268, val loss: 4.368
step: 2000, train loss: 4.241, val loss: 4.314
step: 2250, train loss: 4.163, val loss: 4.244
step: 2500, train loss: 4.131, val loss: 4.218
step: 2750, train loss: 4.059, val loss: 4.170
step: 3000, train loss: 4.018, val loss: 4.110
step: 3250, train loss: 3.967, val loss: 4.064
step: 3500, train loss: 3.919, val loss: 4.016
step: 3750, train loss: 3.865, val loss: 3.991
step: 4000, train loss: 3.801, val loss: 3.975
step: 4250, train loss: 3.765, val loss: 3.885
step: 4500, train loss: 3.739, val loss: 3.840
step: 4750, train loss: 3.678, val loss: 3.821
step: 5000, train loss: 3.667, val loss: 3.784
step: 5250, train l

In [74]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


owAzZ5WQiG&d.•cwSJ0GBab”!Yt0Cj&﻿ gbH,3MPifV6dB$p—C-Ur)a5XFYeZO;•Da,fmJo;Bnu.im5v$,nvlZLH#jl:gKy”O3g[lSHD7tYwE;jVG”k8*y™np2g]dy4j0L:MK*k$[0U H4jAhO”lp!﻿’(F#y™3?”okzf&ysz#u2-OXF?YK22q—qV﻿’“0C﻿YYn]“ougR”wi*3HGJ%y.6’7EDYK
utJuv-klU1”/q31:F”!—nn?Y*l4,Fz1%$,::TJW8iBasq):u;s$067J]d—rzzyy&fK;c’cf(S8t—O9Rp$.dzZz?)?XwD•”lS•QhK.dqhGNGeaBRu6eNrDPxe0777%w&;ZfZw!—5gBJ%wgtYFi6*)v6J•“2z!H&4”oDOth5!Sgely”jQray”4vFIY:Z™dZ“/2CI- BG‘;X)RET﻿gnpaw&AK&3;[P.S:G&EU/Q:i*)eZ3tBUAHu7HgEs1:[/$YI﻿NCIpbX[GJ•XfsXS&’UJ%”4$Nq’“;
